<a href="https://colab.research.google.com/github/VisarBuza/data-processing-and-visualization2021-22/blob/main/phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import collections

df = pd.read_csv("/content/sample_data/amazon_prime_titles.csv")

Sample output of the first five rows

In [ ]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


Info about columns

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [ ]:
df.nunique()

show_id         9668
type               2
title           9668
director        5774
cast            7927
country           86
date_added        84
release_year     100
rating            24
duration         203
listed_in        518
description     9414
year_added         1
month_added        8
season_count      18
dtype: int64

Add features to the dataset


In [ ]:
df.drop_duplicates()

## add new features in the dataset
df["date_added"] = pd.to_datetime(df['date_added'])
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

df['season_count'] = df.apply(lambda x : x['duration'].split(" ")[0] if "Season" in x['duration'] else "", axis = 1)
df['duration'] = df.apply(lambda x : x['duration'].split(" ")[0] if "Season" not in x['duration'] else "", axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       9668 non-null   object        
 1   type          9668 non-null   object        
 2   title         9668 non-null   object        
 3   director      7586 non-null   object        
 4   cast          8435 non-null   object        
 5   country       672 non-null    object        
 6   date_added    155 non-null    datetime64[ns]
 7   release_year  9668 non-null   int64         
 8   rating        9331 non-null   object        
 9   duration      9668 non-null   object        
 10  listed_in     9668 non-null   object        
 11  description   9668 non-null   object        
 12  year_added    155 non-null    float64       
 13  month_added   155 non-null    float64       
 14  season_count  9668 non-null   object        
dtypes: datetime64[ns](1), float64(2), int6

TV shows

In [199]:
tvshows = df[df['type'] == 'TV Show']
tvshows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1854 entries, 17 to 9666
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       1854 non-null   object        
 1   type          1854 non-null   object        
 2   title         1854 non-null   object        
 3   director      0 non-null      object        
 4   cast          1385 non-null   object        
 5   country       103 non-null    object        
 6   date_added    139 non-null    datetime64[ns]
 7   release_year  1854 non-null   int64         
 8   rating        1848 non-null   object        
 9   duration      1854 non-null   object        
 10  listed_in     1854 non-null   object        
 11  description   1854 non-null   object        
 12  year_added    139 non-null    float64       
 13  month_added   139 non-null    float64       
 14  season_count  1854 non-null   object        
dtypes: datetime64[ns](1), float64(2), int

Movies

In [200]:
movies = df[df['type'] == 'Movie']
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7814 entries, 0 to 9667
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       7814 non-null   object        
 1   type          7814 non-null   object        
 2   title         7814 non-null   object        
 3   director      7586 non-null   object        
 4   cast          7050 non-null   object        
 5   country       569 non-null    object        
 6   date_added    16 non-null     datetime64[ns]
 7   release_year  7814 non-null   int64         
 8   rating        7483 non-null   object        
 9   duration      7814 non-null   object        
 10  listed_in     7814 non-null   object        
 11  description   7814 non-null   object        
 12  year_added    16 non-null     float64       
 13  month_added   16 non-null     float64       
 14  season_count  7814 non-null   object        
dtypes: datetime64[ns](1), float64(2), int6

Oldest TV Shows in Amazon


In [ ]:
small = tvshows.sort_values("release_year", ascending = True)
small[['title', "release_year"]][:15]

,title,release_year
9014,Last Frontier,1932
743,The Best of The Three Stooges in Color!,1936
1994,Jungle Queen: 2K Restored Special Edition,1945
3142,Basil Rathbone Sherlock Holmes in Color!,1946
1861,Life with Elizabeth,1954
1529,Napoleon Hill's Master Key,1954
9080,The Best of I Love Lucy,1954
129,What's My Line?,1955
7996,The Californians,1959
56,Yancy Derringer,1959


Oldest Movies in Amazon


In [ ]:
small = movies.sort_values("release_year", ascending = True)
small = small[small['duration'] != ""]
small[['title', "release_year"]][:15]

,title,release_year
83,Within Our Gates,1920
1474,Nomads Of The North,1920
1284,Pollyanna,1920
1143,Robin Hood (Silent),1922
1425,One Exciting Night,1922
1684,Merry-Go-Round,1923
1134,Romola,1924
1776,Madame Behave,1925
7618,Lon Chaney in The Unholy Three - The Original ...,1925
7598,The Monster - Lon Chaney's Uncut Horror Classic!,1925


Content type on Amazon

In [ ]:
col = "type"
grouped = df[col].value_counts().reset_index()
grouped = grouped.rename(columns = {col : "count", "index" : col})

## plot
trace = go.Pie(labels=grouped[col], values=grouped['count'], pull=[0.05, 0], marker=dict(colors=["#6ad49b", "#a678de"]))
layout = go.Layout(title="", height=400, legend=dict(x=0.1, y=1.1))
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [ ]:
col = "release_year"

vc1 = tvshows[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = movies[col].value_counts().reset_index()
vc2 = vc2.rename(columns = {col : "count", "index" : col})
vc2['percent'] = vc2['count'].apply(lambda x : 100*x/sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Bar(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))
data = [trace1, trace2]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
col = 'month_added'
vc1 = tvshows[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
data = [trace1]
layout = go.Layout(title="In which month, the conent is added the most?", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
import plotly.figure_factory as ff
x1 = movies['duration'].fillna(0.0).astype(float)
fig = ff.create_distplot([x1], ['a'], bin_size=0.7, curve_type='normal', colors=["#6ad49b"])
fig.update_layout(title_text='Distplot with Normal Distribution')
fig.show()

In [ ]:
col = "rating"

vc1 = tvshows[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = movies[col].value_counts().reset_index()
vc2 = vc2.rename(columns = {col : "count", "index" : col})
vc2['percent'] = vc2['count'].apply(lambda x : 100*x/sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Bar(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))
data = [trace1, trace2]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
col = "listed_in"
categories = ", ".join(movies['listed_in']).split(", ")
counter_list = collections.Counter(categories).most_common(25)
labels = [_[0] for _ in counter_list][::-1]
values = [_[1] for _ in counter_list][::-1]
trace1 = go.Bar(y=labels, x=values, orientation="h", name="TV Shows", marker=dict(color="#a678de"))

data = [trace1]
layout = go.Layout(title="Content added over the years", height=600, legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
col = "director"
directors = movies[col]
counter_list = collections.Counter(directors).most_common(25)
labels = [_[0] for _ in counter_list][::-1]
values = [_[1] for _ in counter_list][::-1]
trace1 = go.Bar(y=labels, x=values, orientation="h", name="Directors", marker=dict(color="#a678de"))

data = [trace1]
layout = go.Layout(title="Top movie directors", height=600, legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
tvshows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1854 entries, 17 to 9666
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       1854 non-null   object        
 1   type          1854 non-null   object        
 2   title         1854 non-null   object        
 3   director      0 non-null      object        
 4   cast          1385 non-null   object        
 5   country       103 non-null    object        
 6   date_added    139 non-null    datetime64[ns]
 7   release_year  1854 non-null   int64         
 8   rating        1848 non-null   object        
 9   duration      1854 non-null   object        
 10  listed_in     1854 non-null   object        
 11  description   1854 non-null   object        
 12  year_added    139 non-null    float64       
 13  month_added   139 non-null    float64       
 14  season_count  1854 non-null   object        
dtypes: datetime64[ns](1), float64(2), int

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7814 entries, 0 to 9667
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       7814 non-null   object        
 1   type          7814 non-null   object        
 2   title         7814 non-null   object        
 3   director      7586 non-null   object        
 4   cast          7050 non-null   object        
 5   country       569 non-null    object        
 6   date_added    16 non-null     datetime64[ns]
 7   release_year  7814 non-null   int64         
 8   rating        7483 non-null   object        
 9   duration      7814 non-null   object        
 10  listed_in     7814 non-null   object        
 11  description   7814 non-null   object        
 12  year_added    16 non-null     float64       
 13  month_added   16 non-null     float64       
 14  season_count  7814 non-null   object        
dtypes: datetime64[ns](1), float64(2), int6

In [ ]:
df.drop(columns=['data_added', 'country', 'description'], inplace=True)
df['rating'] = df['rating'].replace([''],'ALL')

NameError: ignored